# Omnisci Runtime UDF support

The [*Remote Backend Compiler* (RBC)](https://github.com/xnd-project/rbc) package implements the [Omnisci](https://www.omnisci.com/) client support for defining so-called Runtime UDFs. That is, while Omnisci server is running, one can register new SQL functions to Omnisci Calcite server as well as provide their implementations in LLVM IR string form. The RBC package supports creating Runtime UDFs from Python functions.

One can use the Runtime UDF functions in SQL queries from several Omnisci clients including [`pymapd`](https://github.com/omnisci/pymapd), [`ibis.omniscidb`](http://ibis-project.org/docs/backends/omnisci.html), or [`rbc.omniscidb`](https://github.com/xnd-project/rbc). 

First, we will connect RBC to our local Omnisci server:

In [ ]:
from rbc.omniscidb import RemoteOmnisci
omni = RemoteOmnisci(user='admin', password='HyperInteractive',
                     host='127.0.0.1', port=6274, dbname='omnisci')



In this demo, we'll use the [`ibis.omniscidb`](http://ibis-project.org/docs/backends/omnisci.html) to connect to the OmnisciDB server as well:

In [ ]:
import ibis
ibis_con = ibis.omniscidb.connect(user=omni.user, password=omni.password,
                                  host=omni.host, port=omni.port, database=omni.dbname)

### Create and fill test table

In [ ]:
omni.sql_execute('drop table if exists mytable')
omni.sql_execute('create table if not exists mytable (x DOUBLE, i INT);');
for _i in range(5):
    omni.sql_execute('insert into mytable values '+str((_i,)*2));
ibis_con.sql('select x, i from mytable').execute()

## Defining Runtime UDFs from Python

To define a Runtime UDF from a Python function, the function must be decorated with `omni` decorator:

In [ ]:
@omni('f32(f32)', 'f64(f64)')
def incr(v):
    """Increment float value by one"""
    return v + 1

One can overload existing UDFs (now using Python annotations):

In [ ]:
@omni
def incr(v: 'int32') -> 'int32':
    """Increment integer value by ten"""
    return v + 10

Finally, we register the defined UDFs to OmnisciDB server:

In [ ]:
omni.register()

## Using UDFs in a SQL query

In [ ]:
t = ibis_con.sql('select x, incr(x), i, incr(i) from mytable')
t[t.i < 3].execute()

# Advanced: defining UDFs from a LLVM IR string

One can implement UDF registration support to any OmnisciDB client that 
is able to provide the UDF implementations in LLVM IR form.

To demonstrate that, let's define such a UDF from a LLVM IR string:

In [ ]:
foo_ir = '''
; foo(i, j) -> i * j + 55
define i32 @foo(i32 %.1, i32 %.2) {
entry:
  %.18.i = mul i32 %.2, %.1
  %.33.i = add i32 %.18.i, 55
  ret i32 %.33.i
}
'''
foo_signature = "foo 'int32(int32, int32)'"

and register it to OmnisciDB server using its Thrift end-point:

In [ ]:
omni.thrift_call('register_runtime_udf',
                 omni.session_id,
                 foo_signature,
                 dict(cpu = foo_ir))

Test it:

In [ ]:
ibis_con.sql('select i, foo(i, 5) from mytable').execute()